In [1]:
import pandas as pd 
import numpy as np 
import os 

In [2]:
os.getcwd()

'C:\\Users\\mmccu\\Box\\MM_Personal\\5_Projects\\BoveLab\\3_Data_and_Code\\hva_code\\HomeVideoAnalysis\\megan_sandbox'

In [3]:
# input 
# load mp, mp_world, and yolo .csv files for one video (from main branch, output of home video analysis run.py 8/29/2024)
mp_pose_filepath = r'..\temp\main_branch_outputs\000_run\DS_HC_gait_vertical_left_mediapipe.csv'
mp_world_filepath = r'..\temp\main_branch_outputs\000_run\DS_HC_gait_vertical_left_mediapipe_world.csv'
yolo_filepath = r'..\temp\main_branch_outputs\000_run\DS_HC_gait_vertical_left_yolo.csv'

# output folder 
output_folder = r'..\temp\test_sandbox_pipeline_outputs\001_merge_mp_yolo_dfs'

In [4]:
# input = mp_pose, mp_world, and yolo filepaths for one video 
def merge_mp_pose_world(mp_pose_filepath, mp_world_filepath, yolo_filepath, output_folder):
    mp_pose_df = pd.read_csv(mp_pose_filepath)
    mp_world_df = pd.read_csv(mp_world_filepath)
    yolo_df = pd.read_csv(yolo_filepath)
    
    # rename mp columns 
    mp_pose_df = mp_pose_df.rename(columns = {"X" : "X_pose", 
                                              "Y" : "Y_pose", 
                                              "Z" : "Z_pose", 
                                              "Unnamed: 0" : "label_num"})

    mp_world_df = mp_world_df.rename(columns = {"X" : "X_world", 
                                                 "Y" : "Y_world", 
                                                 "Z" : "Z_world", 
                                                 "Unnamed: 0" : "label_num"})

    yolo_df = yolo_df.rename(columns = {"Unnamed: 0" : "label_num"})
    

    # merge mp world and pose (same vis, markers)
    mp_all_df = mp_pose_df
    mp_all_df['X_world'] = mp_world_df['X_world']
    mp_all_df['Y_world'] = mp_world_df['Y_world']
    mp_all_df['Z_world'] = mp_world_df['Z_world']
    
    # take negative of Y values - when Y is negative the "stick figure" plots right side up and is more intuitive for gait calculations   
    mp_all_df.loc[:,'Y_pose_negative'] = -mp_all_df['Y_pose']
    mp_all_df.loc[:,'Y_world_negative'] = -(mp_all_df['Y_world'])
    yolo_df.loc[:,'Y_negative'] = -(yolo_df['Y'])

    # save merged mp df as csv
    input_file_no_ext_1 = os.path.splitext(os.path.basename(mp_pose_filepath))[0]
    output_file_name_1 = input_file_no_ext_1 + '_all.csv'
    output_file_1 = os.path.normpath(os.path.join(output_folder, output_file_name_1))
    mp_all_df.to_csv(output_file_1)
    
    # save yolo df as csv  
    input_file_no_ext_2 = os.path.splitext(os.path.basename(yolo_filepath))[0]
    output_file_name_2 = input_file_no_ext_2 + '.csv'
    output_file_2 = os.path.normpath(os.path.join(output_folder, output_file_name_2))
    yolo_df.to_csv(output_file_2)

    return([mp_all_df, yolo_df])

In [5]:
[mp_all_df, yolo_df] = merge_mp_pose_world(mp_pose_filepath, mp_world_filepath, yolo_filepath, output_folder)

print(mp_all_df.head())
print(yolo_df.head())

   label_num    X_pose    Y_pose    Z_pose       vis  pres  frame  \
0          0  0.564727 -0.231156  0.797350  0.996966   0.0      0   
1          1  0.538995 -0.261305  0.635333  0.997083   0.0      0   
2          2  0.520686 -0.260609  0.634956  0.997018   0.0      0   
3          3  0.502225 -0.259820  0.634332  0.997107   0.0      0   
4          4  0.588311 -0.263609  0.624654  0.996420   0.0      0   

             label   X_world   Y_world   Z_world  Y_pose_negative  \
0             nose  0.011313 -0.644613  0.129704         0.231156   
1   left_eye_inner -0.013144 -0.671381  0.098242         0.261305   
2         left_eye -0.013018 -0.671677  0.098631         0.260609   
3   left_eye_outer -0.013117 -0.671975  0.097893         0.259820   
4  right_eye_inner  0.018931 -0.671060  0.097628         0.263609   

   Y_world_negative  
0          0.644613  
1          0.671381  
2          0.671677  
3          0.671975  
4          0.671060  
   label_num    X    Y      label  fra